In [1]:
import sys, os

ms_udfix = lambda x:x.replace('\\', '/')[:-1]

sys.path.append("samples/coco/")
from coco import *


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\pycocotools\coco.py:49: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  import matplotlib; matplotlib.use('Agg')
Using TensorFlow backend.


In [2]:
train_ud = ms_udfix(r"C:\Users\Public\Segmentation\seg-countertops\annotationsWithImages\\")
V_ROOT_DIR = ms_udfix(r"C:\Users\Public\Mask_RCNN-master\\")

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ms_udfix(r"C:\Users\Public\Segmentation\seg-countertops\\"), "logs/")
if not(os.path.exists(MODEL_DIR)):os.mkdir(MODEL_DIR)
#MODEL_DIR = os.path.join(ms_udfix(r"C:\Users\Public\Mask_RCNN-master\\"), "logs")

logs = os.path.join(ms_udfix(r"C:\Users\dominics.CORP\Documents\dev\MaskRCNN_ds\model_logs\\"))
if not(os.path.exists(logs)):
    os.mkdir(logs)

# Path to trained weights file
# Download this file and place in the root of your 
# project (See README file for details)
#You should have trained with this: COCO_MODEL_PATH = "C:\Users\Public\Mask_RCNN-master\mask_rcnn_coco.h5")
#COCO_MODEL_PATH = os.path.join(V_ROOT_DIR, "mask_rcnn_coco1_0006.h5")
COCO_MODEL_PATH = os.path.join(V_ROOT_DIR, "mask_rcnn_coco.h5")

#after 10cycles of Training network heads
#COCO_MODEL_PATH = ms_udfix(r"C:\Users\dominics.CORP\Documents\dev\MaskRCNN_ds\model_logs\coco20181105T1327\mask_rcnn_coco_0010.h5 ")
# Directory of images to run detection on
IMAGE_DIR = train_ud##os.path.join(ROOT_DIR, "graniteimages")

In [3]:
args = """python coco.py train --dataset=C:/Users/Public/Mask_RCNN-master/segmentation/kitchen/ --model=%s --logs %s"""%(COCO_MODEL_PATH, logs)

sys.argv = args.split(' ')[1:]

args = make_parser()
print("Command: ", args.command)
print("Model: ", args.model)
print("Dataset: ", args.dataset)
print("Year: ", args.year)
print("Logs: ", args.logs)
print("Auto Download: ", args.download)

Command:  train
Model:  C:/Users/Public/Mask_RCNN-master/mask_rcnn_coco.h5
Dataset:  C:/Users/Public/Mask_RCNN-master/segmentation/kitchen/
Year:  NONE
Logs:  C:/Users/dominics.CORP/Documents/dev/MaskRCNN_ds/model_logs/
Auto Download:  False


In [4]:
#Run modificsation:
TRAIN_SUBSET = "1106d_HL_train"
NUM_CLASSES = 4+1


In [5]:
#Load Model for training
config = CocoConfig()
config.IMAGES_PER_GPU = 1
config.BATCH_SIZE=1
config.STEPS_PER_EPOCH=100
config.VALIDATION_STEPS=10
#config.NUM_CLASSES = NUM_CLASSES #SET IN CONFIG.PY LINE 73
config.LOSS_WEIGHTS ={'rpn_bbox_loss': 1.0, 
                      'mrcnn_mask_loss': 4.0, 
                      'rpn_class_loss': 1.0, 
                      'mrcnn_class_loss': 1.0, 
                      'mrcnn_bbox_loss': 1.0}

#config.IMAGE_META_SIZE = 18#HACH 1 + 3 + 3 + 4 + 1 + self.NUM_CLASSES #SET IN CONFIG.PY LINE238

model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=args.logs)

# Select weights file to load
if args.model.lower() == "coco":
    model_path = COCO_MODEL_PATH
elif args.model.lower() == "last":
    # Find last trained weights
    model_path = model.find_last()[1]
elif args.model.lower() == "imagenet":
    # Start from ImageNet trained weights
    model_path = model.get_imagenet_weights()
else:
    model_path = args.model
print(model_path)
# Load weights
print("Loading weights ", model_path)
print('*************')
model.load_weights(model_path, by_name=True,exclude=["mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_bbox", "mrcnn_mask"])
print('###############')

config.display()

C:/Users/Public/Mask_RCNN-master/mask_rcnn_coco.h5
Loading weights  C:/Users/Public/Mask_RCNN-master/mask_rcnn_coco.h5
*************
###############

Configurations:
BACKBONE                       resnet101
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                17
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEAR

In [ ]:
#Train
dataset_train = CocoDataset()
dataset_train.load_coco(args.dataset, TRAIN_SUBSET)

c = 0;
for img in dataset_train.image_info:c+=len(img['annotations'])
print('dataset_train.image_info: [n=%i] total annotations: %i'%(len(dataset_train.image_info), c))
#COCO API consumes data.json [info,categories,annotations,categories] 
#for details: https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocotools/coco.py

dataset_train.prepare()
#dataset_train.load_coco(args.dataset, "val35k")

#Validation dataset
dataset_val = CocoDataset()
dataset_val.load_coco(args.dataset, TRAIN_SUBSET)
dataset_val.prepare()

# *** This training schedule is an example. Update to your needs ***

#Training - Stage 1
print("Training network heads")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=10,
            layers='heads')

# Training - Stage 2
#Finetune layers from ResNet stage 4 and up

print("Fine tune Resnet stage 4 and up")
model.train(dataset_train, dataset_val,
             learning_rate=config.LEARNING_RATE,
             epochs=50,
             layers='4+')

# Training - Stage 3
# Fine tune all layers
# print("Fine tune all layers")
# model.train(dataset_train, dataset_val,
#              learning_rate=config.LEARNING_RATE / 10,
#              epochs=10,
#              layers='all')


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
CocoDataset(utils.Dataset): image_dir: C:/Users/Public/Mask_RCNN-master/segmentation/kitchen/newtrain json_path: annotations/1106d_HL_data.json
dataset_train.image_info: [n=77] total annotations: 398
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
CocoDataset(utils.Dataset): image_dir: C:/Users/Public/Mask_RCNN-master/segmentation/kitchen/newtrain json_path: annotations/1106d_HL_data.json
Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: C:/Users/dominics.CORP/Documents/dev/MaskRCNN_ds/model_logs/coco20181108T1714\mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
100/100 [==============================] - 116s 1s/step - loss: 2.9930 - rpn_class_loss: 0.0306 - rpn_bbox_loss: 0.3399 - mrcnn_class_loss: 0.2404 - mrcnn_bbox_loss: 0.4022 - mrcnn_mask_loss: 1.9799 - val_loss: 2.7116 - val_rpn_class_loss: 0.0254 - val_rpn_bbox_loss: 0.3262 - val_mrcnn_class_loss: 0.1988 - val_mrcnn_bbox_loss: 0.2944 - val_mrcnn_mask_loss: 1.8668
Epoch 2/10
100/100 [==============================] - 107s 1s/step - loss: 2.6764 - rpn_class_loss: 0.0245 - rpn_bbox_loss: 0.2710 - mrcnn_class_loss: 0.2046 - mrcnn_bbox_loss: 0.3211 - mrcnn_mask_loss: 1.8551 - val_loss: 2.4315 - val_rpn_class_loss: 0.0191 - val_rpn_bbox_loss: 0.2170 - val_mrcnn_class_loss: 0.1767 - val_mrcnn_bbox_loss: 0.2941 - val_mrcnn_mask_loss: 1.7245
Epoch 3/10
100/100 [==============================] - 105s 1s/step - loss: 2.4626 - rpn_class_loss: 0.0219 - rpn_bbox_loss: 0.2437 - mrcnn_class_loss: 0.1864 - mrcnn_bbox_loss: 0.2656 - mrcnn_mask_loss: 1.7449 - val_loss: 2.2082 - val_rpn_class_l

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 11/50
100/100 [==============================] - 137s 1s/step - loss: 1.6569 - rpn_class_loss: 0.0149 - rpn_bbox_loss: 0.1017 - mrcnn_class_loss: 0.1221 - mrcnn_bbox_loss: 0.1397 - mrcnn_mask_loss: 1.2785 - val_loss: 1.5433 - val_rpn_class_loss: 0.0119 - val_rpn_bbox_loss: 0.0846 - val_mrcnn_class_loss: 0.1125 - val_mrcnn_bbox_loss: 0.1668 - val_mrcnn_mask_loss: 1.1675
Epoch 12/50
100/100 [==============================] - 121s 1s/step - loss: 1.5151 - rpn_class_loss: 0.0122 - rpn_bbox_loss: 0.0846 - mrcnn_class_loss: 0.1172 - mrcnn_bbox_loss: 0.1464 - mrcnn_mask_loss: 1.1548 - val_loss: 1.4174 - val_rpn_class_loss: 0.0112 - val_rpn_bbox_loss: 0.0697 - val_mrcnn_class_loss: 0.1303 - val_mrcnn_bbox_loss: 0.1091 - val_mrcnn_mask_loss: 1.0971
Epoch 13/50
100/100 [==============================] - 119s 1s/step - loss: 1.3508 - rpn_class_loss: 0.0109 - rpn_bbox_loss: 0.0642 - mrcnn_class_loss: 0.0966 - mrcnn_bbox_loss: 0.1349 - mrcnn_mask_loss: 1.0442 - val_loss: 1.2723 - val_rpn_clas

Epoch 33/50
100/100 [==============================] - 117s 1s/step - loss: 0.2533 - rpn_class_loss: 0.0014 - rpn_bbox_loss: 0.0033 - mrcnn_class_loss: 0.0230 - mrcnn_bbox_loss: 0.0172 - mrcnn_mask_loss: 0.2084 - val_loss: 0.2382 - val_rpn_class_loss: 4.3953e-04 - val_rpn_bbox_loss: 0.0035 - val_mrcnn_class_loss: 0.0150 - val_mrcnn_bbox_loss: 0.0172 - val_mrcnn_mask_loss: 0.2020
Epoch 34/50
100/100 [==============================] - 118s 1s/step - loss: 0.2433 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.0033 - mrcnn_class_loss: 0.0219 - mrcnn_bbox_loss: 0.0176 - mrcnn_mask_loss: 0.1993 - val_loss: 0.2666 - val_rpn_class_loss: 7.4735e-04 - val_rpn_bbox_loss: 0.0028 - val_mrcnn_class_loss: 0.0268 - val_mrcnn_bbox_loss: 0.0157 - val_mrcnn_mask_loss: 0.2205
Epoch 35/50
100/100 [==============================] - 17150s 172s/step - loss: 0.2366 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.0029 - mrcnn_class_loss: 0.0188 - mrcnn_bbox_loss: 0.0162 - mrcnn_mask_loss: 0.1976 - val_loss: 0.2524 - 

In [ ]:
dataset_train.class_info

In [ ]:
dataset_dir=args.dataset
json_path_dict = {
                "1106d_HL_train":   [os.path.join(dataset_dir, "newtrain"), "annotations/1106d_HL_data.json"],
                "train":            [os.path.join(dataset_dir, "newtrain"), "annotations/data.json"],
                "val":              [os.path.join(dataset_dir, "val2014"), "annotations/data.json"],
                "minival":          [os.path.join(dataset_dir, "val2014"), "annotations1/instances_minival2014.json"],
                "val35k":           [os.path.join(dataset_dir, "val2014"), "annotations1/instances_valminusminival2014.json"],
            }
image_dir, json_path = json_path_dict[TRAIN_SUBSET]
coco = COCO(os.path.join(dataset_dir, json_path))
        
#print(dataset_train.num_classes)
class_ids = sorted(coco.getCatIds())


In [ ]:
#class_ids
#coco.getImgIds(catIds=[4])
class_ids = set(sorted(coco.getCatIds()))
print(class_ids)

In [ ]:
import tensorflow as tf
from tensorflow.python.ops.variables import Variable

for k, v in locals().items():
    if type(v) !=str and type(v) !=list: print(k, type(v), v)
    if type(v) is Variable or type(v) is tf.Tensor:
        print("{0}: {1}".format(k, v))

In [ ]:
# Validation dataset

dataset_val = CocoDataset()
coco = dataset_val.load_coco(args.dataset, "minival", return_coco=True)
dataset_val.prepare()
print("Running COCO evaluation on {} images.".format(args.limit))
evaluate_coco(model, dataset_val, coco, "bbox", limit=int(args.limit))